In [1]:
#basic
import os
import os.path as op

import numpy as np
import pandas as pd
import xarray as xr
import datetime

#plots
from ipyleaflet import Map, TileLayer, basemaps, Heatmap
from ipyleaflet.velocity import Velocity
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
from ipywidgets import interactive

In [2]:
date = datetime.date.today() # today or ex: '20200707'
date = date.strftime("%Y%m%d")
url  = 'https://nomads.ncep.noaa.gov/dods/wave/mww3/'+date+'/multi_1.glo_30mext'+date+'_00z'
lat = 44.0
lon = -3.6
data = xr.open_dataset(url)
data['uwavedir'] = data['htsgwsfc']*np.cos(data['dirpwsfc']*np.pi/180)
data['vwavedir'] = data['htsgwsfc']*np.sin(data['dirpwsfc']*np.pi/180)

In [3]:
def plot_forecast(time):
    variable = data['htsgwsfc'].sel(time=time).to_dataframe().reset_index()
    fig = go.Figure(go.Densitymapbox(lat=variable.lat, lon=variable.lon, z=variable.htsgwsfc,
                                     radius=9, colorscale='Jet'))
    fig.update_layout(
        mapbox_zoom=2,
        mapbox_style="white-bg",
        mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }],
        mapbox_center_lon=lon,
        mapbox_center_lat=lat)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

interactive_plot = interactive(plot_forecast, 
                               time=data.time.values)
interactive_plot

interactive(children=(Dropdown(description='time', options=(numpy.datetime64('2021-01-19T00:00:00.000000000'),…

In [ ]:
center = [lat, lon]
zoom = 5
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.Esri.WorldImagery)

display_options = {
    'velocityType': 'Global Wave',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wave data'
}

wave = Velocity(data=data.isel(time=0),
                zonal_speed='uwavedir',
                meridional_speed='vwavedir',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=float(np.nanmax(data.isel(time=0).htsgwsfc.values)),
                display_options=display_options)
m.add_layer(wave)

m

In [ ]:
center = [lat, lon]
zoom = 5
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.Esri.WorldImagery)

display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}

wind = Velocity(data=data.isel(time=0),
                zonal_speed='ugrdsfc',
                meridional_speed='vgrdsfc',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=np.nanmax(data.isel(time=0).windsfc.values)-5,
                display_options=display_options)
m.add_layer(wind)

m

In [ ]:
surfbreaks = pd.read_csv('../data/surfbreaks.csv')

mapbox_access_token = open("../.mapbox_token").read()

fig = go.Figure(go.Scattermapbox(
        lat=surfbreaks.latitude.to_list(),
        lon=surfbreaks.longitude.to_list(),
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=9,
            color='red'
        ),
        text=surfbreaks.surfbreak.to_list(),
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style='satellite-streets',
        bearing=0,
        center=dict(
            lat=43.47,
            lon=-3.78
        ),
        pitch=0,
        zoom=8
    ),
)

fig.show()

In [ ]:
df = data.isel(lon=np.argmin(np.abs(data.lon.values-(360+lon))))\
.isel(lat=np.argmin(np.abs(data.lat.values-lat))).to_dataframe().reset_index()

In [ ]:
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df["time"],
        y=df["htsgwsfc"],
        mode="lines",
        name="Hs"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["time"],
        y=df["perpwsfc"],
        mode="lines",
        name="Tp"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["time"],
        y=df["dirpwsfc"],
        mode="markers",
        name="Dir"
    ),
    row=4, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Time", "Significant wave<br>height, Hs", "Peak<br>Period, Tp",
                    "Mean wave<br>direction, Dir"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df['time'].tolist(), df['htsgwsfc'].tolist(), 
                    df['perpwsfc'].tolist(), df['dirpwsfc'].tolist()],
            align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    width=900,
    height=1000,
    showlegend=False,
    title_text="Principal variables of the forecast",
)

fig.show()